# User-player regression
Regressions targeting sentiment with covariates from both users and players. Single rows are determined by user-player-year triplet key

### Imports / load

In [1]:
import pandas as pd
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

C:\Users\map22\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
nba_df = pd.read_csv('c:/Users/map22/Google Drive/sentiment_nba/nba_user_player_sentiment.tsv', sep='\t')
nba_df = nba_df.dropna(subset=['Race', 'PPG']) # get some name matches for years players weren't playing / coaches
nba_df['rookie'] = nba_df['experience']<=1
nba_df = nba_df.replace({'M':'B', 'L':'W'}).query('Race == "W" or Race == "B"')

C:\Users\map22\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


#### Get salary residuals for NBA

In [3]:
nba_cov_df = pd.read_csv('modeling_data/nba_model_data.tsv', sep='\t')
nba_cov_df['rookie_contract'] = nba_cov_df['experience'] <=4

In [4]:
salary_model = smf.wls( formula = 'standard_salary ~ rookie + MP + PPG + ThreePP + DWS + TRBP * height_dummies + AST + BLKP * height_dummies + STLP + TOVP',
                data = nba_cov_df, weights = nba_cov_df['G']).fit()

In [5]:
nba_cov_df['pred_std_salary'] = salary_model.predict(nba_cov_df)
nba_cov_df['std_salary_resid'] = nba_cov_df['pred_std_salary']- nba_cov_df['standard_salary']
nba_cov_df.loc[ nba_cov_df['G'] <20, 'std_salary_resid'] = 0

In [6]:
nba_df = nba_df.merge(nba_cov_df[['Player','year', 'std_salary_resid']],
                      on=['Player', 'year'], how = 'left')

In [7]:
for col in ['FTr','TOVP', 'standard_salary', 'std_salary_resid', 'clinton_vote_lead', 'white_black_diff','total_population' ]:
    nba_df[col] = nba_df[col].fillna(nba_df[col].mean())

#### NFL

In [8]:
nfl_df = pd.read_csv('c:/Users/map22/Google Drive/sentiment_nba/nfl_user_player_sentiment.tsv', sep='\t')
nfl_df = nfl_df.dropna(subset=['race']) # get some name matches for years players weren't playing / coaches
nfl_df['rookie'] = nfl_df['experience'] <=1
nfl_df['race'] = nfl_df['race'].replace({'L':'B', 'S':'B', 'M':'B'})

In [9]:
nfl_cov_df = pd.read_csv('modeling_data/nfl_model_data.tsv', sep='\t')
nfl_cov_df['rookie_contract'] = nfl_cov_df['experience'] <=4

In [10]:
salary_model = smf.wls( formula = 'standard_salary ~ rookie_contract + position + z_DVOA',
                data = nfl_cov_df, weights = 1).fit()

In [14]:
nfl_cov_df['pred_std_salary'] = salary_model.predict(nfl_cov_df)
nfl_cov_df['std_salary_resid'] = nfl_cov_df['pred_std_salary']- nfl_cov_df['standard_salary']

In [15]:
nfl_df = nfl_df.merge(nfl_cov_df[['Player','year', 'std_salary_resid']],
                      on=['Player', 'year'], how = 'left')

In [18]:
for col in [ 'std_salary_resid', 'clinton_vote_lead', 'white_black_diff','total_population' ]:
    nfl_df[col] = nfl_df[col].fillna(nfl_df[col].mean())

## NBA
#### Performance only model

In [48]:
nba_df['demean_PPG'] = nba_df['PPG'] - nba_df['PPG'].mean()
nba_df['demean_clinton'] = nba_df['clinton_vote_lead'] - nba_df['clinton_vote_lead'].mean()
nba_df['demean_race_diff'] = nba_df['white_black_diff'] - nba_df['white_black_diff'].mean()

In [18]:
model = smf.wls( formula = 'compound_mean ~ MP + PPG', \
#                       ' PPG +  + total_population+  * white_black_diff + C(Race) * clinton_vote_lead',
                data = nba_df,
               weights = 1,# / (nba_df['compound_mean_std'] / np.sqrt(fit_df['user_count'])),
#                missing='raise'   
               ).fit()

In [19]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:          compound_mean   R-squared:                       0.000
Model:                            WLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     99.48
Date:                Mon, 24 Dec 2018   Prob (F-statistic):           6.33e-44
Time:                        06:12:21   Log-Likelihood:            -2.8257e+05
No. Observations:              886736   AIC:                         5.652e+05
Df Residuals:                  886733   BIC:                         5.652e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0702      0.001     76.323      0.000       0.068       0.072
MP         -7.649e-08   6.34e-07     -0.121      0.904   -1.32e-06    1.17e-06
PPG            0.0006   6.46e-05      9.796      0.000       0.001       0.001
==============================================================================
Omnibus:                     1095.622   Durbin-Watson:                   1.477
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1225.433
Skew:                          -0.045   Prob(JB):                    7.95e-267
Kurtosis:                       3.158   Cond. No.                     5.44e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.44e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

#### Full performance

In [20]:
model = smf.wls( formula = 'compound_mean ~ MP + PPG + ThreePP + DWS + TRBP + FTr + ASTP + BLKP + STLP + TOVP', \
#                       ' PPG +  + total_population+  * white_black_diff + C(Race) * clinton_vote_lead',
                data = nba_df,
               weights = 1,# / (nba_df['compound_mean_std'] / np.sqrt(fit_df['user_count'])),
#                missing='raise'   
               ).fit()

In [21]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:          compound_mean   R-squared:                       0.001
Model:                            WLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     55.53
Date:                Mon, 24 Dec 2018   Prob (F-statistic):          7.03e-113
Time:                        06:14:20   Log-Likelihood:            -2.8222e+05
No. Observations:              886123   AIC:                         5.645e+05
Df Residuals:                  886112   BIC:                         5.646e+05
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0803      0.002     33.625      0.000       0.076       0.085
MP          5.631e-07   8.74e-07      0.644      0.520   -1.15e-06    2.28e-06
PPG            0.0007   9.02e-05      7.922      0.000       0.001       0.001
ThreePP        0.0099      0.003      3.024      0.002       0.003       0.016
DWS           -0.0006      0.001     -1.147      0.252      -0.002       0.000
TRBP          -0.0012      0.000     -9.833      0.000      -0.001      -0.001
FTr           -0.0066      0.003     -2.409      0.016      -0.012      -0.001
ASTP          -0.0001   5.02e-05     -2.484      0.013      -0.000   -2.63e-05
BLKP           0.0028      0.000      8.628      0.000       0.002       0.003
STLP           0.0029      0.001      4.907      0.000       0.002       0.004
TOVP          -0.0005      0.000     -5.031      0.000      -0.001      -0.000
==============================================================================
Omnibus:                     1082.997   Durbin-Watson:                   1.478
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1210.575
Skew:                          -0.045   Prob(JB):                    1.34e-263
Kurtosis:                       3.158   Cond. No.                     2.14e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.14e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

#### Full performance + height / age

In [45]:
nfl_df['demean_DVOA'] = nfl_df['z_DVOA'] - nfl_df['z_DVOA'].mean()
nfl_df['demean_clinton'] = nfl_df['clinton_vote_lead'] - nfl_df['clinton_vote_lead'].mean()
nfl_df['demean_race_diff'] = nfl_df['white_black_diff'] - nfl_df['white_black_diff'].mean()

In [27]:
model = smf.wls( formula = 'compound_mean ~ MP + PPG + ThreePP + DWS + TRBP + FTr + ASTP + BLKP + STLP + TOVP' \
                       ' + C(height_dummies) + rookie + youth + oldness',
                data = nba_df,
               weights = 1,# / (nba_df['compound_mean_std'] / np.sqrt(fit_df['user_count'])),
#                missing='raise'   
               ).fit()

In [28]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:          compound_mean   R-squared:                       0.001
Model:                            WLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     81.82
Date:                Mon, 24 Dec 2018   Prob (F-statistic):          3.30e-252
Time:                        07:26:49   Log-Likelihood:            -2.8188e+05
No. Observations:              886123   AIC:                         5.638e+05
Df Residuals:                  886107   BIC:                         5.640e+05
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept                      0.0660      0.002     26.496      0.000       0.061       0.071
C(height_dummies)[T.short]     0.0108      0.002      5.055      0.000       0.007       0.015
C(height_dummies)[T.tall]     -0.0020      0.001     -1.922      0.055      -0.004    3.84e-05
rookie[T.True]                 0.0148      0.001     10.022      0.000       0.012       0.018
MP                         -1.574e-06   8.87e-07     -1.774      0.076   -3.31e-06    1.65e-07
PPG                            0.0011   9.36e-05     12.094      0.000       0.001       0.001
ThreePP                        0.0125      0.003      3.792      0.000       0.006       0.019
DWS                            0.0009      0.001      1.831      0.067   -6.68e-05       0.002
TRBP                          -0.0010      0.000     -7.878      0.000      -0.001      -0.001
FTr                           -0.0041      0.003     -1.475      0.140      -0.010       0.001
ASTP                          -0.0002   5.27e-05     -3.447      0.001      -0.000   -7.83e-05
BLKP                           0.0019      0.000      5.679      0.000       0.001       0.002
STLP                           0.0016      0.001      2.571      0.010       0.000       0.003
TOVP                          -0.0005      0.000     -4.328      0.000      -0.001      -0.000
youth                          0.0027      0.000     11.443      0.000       0.002       0.003
oldness                        0.0019      0.000     12.333      0.000       0.002       0.002
==============================================================================
Omnibus:                     1072.458   Durbin-Watson:                   1.479
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1200.485
Skew:                          -0.044   Prob(JB):                    2.08e-261
Kurtosis:                       3.158   Cond. No.                     2.17e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.17e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

#### Full performance + height / age + salary

In [19]:
model = smf.wls( formula = 'compound_mean ~ MP + PPG + ThreePP + DWS + TRBP + FTr + ASTP + BLKP + STLP + TOVP' \
                       ' + C(height_dummies) + rookie + youth + oldness + standard_salary + std_salary_resid',
                data = nba_df,
               weights = 1,# / (nba_df['compound_mean_std'] / np.sqrt(fit_df['user_count'])),
#                missing='raise'   
               ).fit(cov_type='cluster', cov_kwds={'groups' : nba_df['Player']})

In [20]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:          compound_mean   R-squared:                       0.001
Model:                            WLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     6.232
Date:                Mon, 24 Dec 2018   Prob (F-statistic):           6.68e-14
Time:                        08:33:11   Log-Likelihood:            -2.7989e+05
No. Observations:              880544   AIC:                         5.598e+05
Df Residuals:                  880526   BIC:                         5.600e+05
Df Model:                          17                                         
Covariance Type:              cluster                                         
==============================================================================================
                                 coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept                      0.0632      0.013      4.918      0.000       0.038       0.088
C(height_dummies)[T.short]     0.0115      0.010      1.137      0.256      -0.008       0.031
C(height_dummies)[T.tall]     -0.0019      0.005     -0.373      0.709      -0.012       0.008
rookie[T.True]                 0.0146      0.005      2.975      0.003       0.005       0.024
MP                         -1.185e-06   2.99e-06     -0.396      0.692   -7.05e-06    4.68e-06
PPG                            0.0011      0.000      2.241      0.025       0.000       0.002
ThreePP                        0.0124      0.014      0.875      0.382      -0.015       0.040
DWS                            0.0007      0.002      0.375      0.707      -0.003       0.005
TRBP                          -0.0010      0.001     -2.014      0.044      -0.002   -2.76e-05
FTr                           -0.0042      0.009     -0.468      0.640      -0.022       0.014
ASTP                          -0.0002      0.000     -0.923      0.356      -0.001       0.000
BLKP                           0.0020      0.001      1.512      0.130      -0.001       0.005
STLP                           0.0019      0.002      0.833      0.405      -0.003       0.006
TOVP                          -0.0004      0.000     -1.289      0.197      -0.001       0.000
youth                          0.0029      0.001      3.080      0.002       0.001       0.005
oldness                        0.0019      0.001      2.561      0.010       0.000       0.003
standard_salary             2.375e-05      0.000      0.089      0.929      -0.000       0.001
std_salary_resid            -6.22e-05      0.000     -0.218      0.827      -0.001       0.000
==============================================================================
Omnibus:                     1065.698   Durbin-Watson:                   1.478
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1193.334
Skew:                          -0.044   Prob(JB):                    7.43e-260
Kurtosis:                       3.158   Cond. No.                     2.74e+04
==============================================================================

Warnings:
[1] Standard Errors are robust tocluster correlation (cluster)
[2] The condition number is large, 2.74e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

#### Full performance + height / age + salary + Team wins

In [74]:
model = smf.wls( formula = 'compound_mean ~ MP + PPG + ThreePP + DWS + TRBP + FTr + ASTP + BLKP + STLP + TOVP' \
                       ' + C(height_dummies) + rookie + youth + oldness + standard_salary + std_salary_resid + Wins',
                data = nba_df,
               weights = 1,# / (nba_df['compound_mean_std'] / np.sqrt(fit_df['user_count'])),
#                missing='raise'   
               ).fit(cov_type='cluster', cov_kwds={'groups' : nba_df['Player']})

In [75]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:          compound_mean   R-squared:                       0.001
Model:                            WLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     5.980
Date:                Mon, 24 Dec 2018   Prob (F-statistic):           8.53e-14
Time:                        08:02:03   Log-Likelihood:            -2.7989e+05
No. Observations:              880544   AIC:                         5.598e+05
Df Residuals:                  880525   BIC:                         5.600e+05
Df Model:                          18                                         
Covariance Type:              cluster                                         
==============================================================================================
                                 coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept                      0.0678      0.014      5.010      0.000       0.041       0.094
C(height_dummies)[T.short]     0.0115      0.010      1.143      0.253      -0.008       0.031
C(height_dummies)[T.tall]     -0.0020      0.005     -0.400      0.689      -0.012       0.008
rookie[T.True]                 0.0146      0.005      2.977      0.003       0.005       0.024
MP                         -1.924e-06   3.22e-06     -0.597      0.550   -8.24e-06    4.39e-06
PPG                            0.0011      0.001      2.164      0.030       0.000       0.002
ThreePP                        0.0129      0.014      0.903      0.367      -0.015       0.041
DWS                            0.0016      0.002      0.683      0.495      -0.003       0.006
TRBP                          -0.0011      0.001     -2.075      0.038      -0.002   -6.08e-05
FTr                           -0.0042      0.009     -0.470      0.639      -0.022       0.013
ASTP                          -0.0002      0.000     -0.925      0.355      -0.001       0.000
BLKP                           0.0020      0.001      1.522      0.128      -0.001       0.005
STLP                           0.0016      0.002      0.733      0.464      -0.003       0.006
TOVP                          -0.0004      0.000     -1.300      0.194      -0.001       0.000
youth                          0.0028      0.001      3.139      0.002       0.001       0.005
oldness                        0.0018      0.001      2.523      0.012       0.000       0.003
standard_salary             4.223e-05      0.000      0.160      0.873      -0.000       0.001
std_salary_resid           -4.326e-05      0.000     -0.155      0.877      -0.001       0.001
Wins                        -9.04e-05      0.000     -0.603      0.547      -0.000       0.000
==============================================================================
Omnibus:                     1065.367   Durbin-Watson:                   1.478
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1192.863
Skew:                          -0.044   Prob(JB):                    9.40e-260
Kurtosis:                       3.158   Cond. No.                     2.89e+04
==============================================================================

Warnings:
[1] Standard Errors are robust tocluster correlation (cluster)
[2] The condition number is large, 2.89e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

#### Full performance + height / age + salary + race of player

In [71]:
model = smf.wls( formula = 'compound_mean ~ MP + PPG + ThreePP + DWS + TRBP + FTr + ASTP + BLKP + STLP + TOVP' \
                       ' + C(height_dummies) + rookie + youth + oldness + standard_salary + std_salary_resid + C(Race)',
                data = nba_df,
               weights = 1,# / (nba_df['compound_mean_std'] / np.sqrt(fit_df['user_count'])),
#                missing='raise'   
               ).fit(cov_type='cluster', cov_kwds={'groups' : nba_df['Player']})

In [72]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:          compound_mean   R-squared:                       0.001
Model:                            WLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     6.556
Date:                Mon, 24 Dec 2018   Prob (F-statistic):           1.67e-15
Time:                        07:56:00   Log-Likelihood:            -2.7988e+05
No. Observations:              880544   AIC:                         5.598e+05
Df Residuals:                  880525   BIC:                         5.600e+05
Df Model:                          18                                         
Covariance Type:              cluster                                         
==============================================================================================
                                 coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept                      0.0623      0.012      4.996      0.000       0.038       0.087
C(height_dummies)[T.short]     0.0120      0.010      1.190      0.234      -0.008       0.032
C(height_dummies)[T.tall]     -0.0026      0.005     -0.499      0.617      -0.013       0.008
rookie[T.True]                 0.0143      0.005      2.921      0.003       0.005       0.024
C(Race)[T.W]                   0.0054      0.007      0.722      0.471      -0.009       0.020
MP                          -1.21e-06   2.96e-06     -0.409      0.683   -7.02e-06     4.6e-06
PPG                            0.0012      0.001      2.330      0.020       0.000       0.002
ThreePP                        0.0104      0.015      0.692      0.489      -0.019       0.040
DWS                            0.0007      0.002      0.364      0.716      -0.003       0.005
TRBP                          -0.0011      0.001     -2.090      0.037      -0.002   -6.68e-05
FTr                           -0.0048      0.009     -0.548      0.584      -0.022       0.012
ASTP                          -0.0002      0.000     -0.935      0.350      -0.001       0.000
BLKP                           0.0022      0.001      1.759      0.079      -0.000       0.005
STLP                           0.0022      0.002      0.999      0.318      -0.002       0.006
TOVP                          -0.0004      0.000     -1.260      0.208      -0.001       0.000
youth                          0.0029      0.001      3.127      0.002       0.001       0.005
oldness                        0.0018      0.001      2.538      0.011       0.000       0.003
standard_salary             1.161e-05      0.000      0.044      0.965      -0.001       0.001
std_salary_resid              -0.0001      0.000     -0.364      0.716      -0.001       0.000
==============================================================================
Omnibus:                     1067.670   Durbin-Watson:                   1.478
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1195.438
Skew:                          -0.044   Prob(JB):                    2.59e-260
Kurtosis:                       3.158   Cond. No.                     2.74e+04
==============================================================================

Warnings:
[1] Standard Errors are robust tocluster correlation (cluster)
[2] The condition number is large, 2.74e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

#### Full performance + height / age + salary + race of player

In [50]:
model = smf.wls( formula = 'compound_mean ~ MP + ThreePP + DWS + TRBP + FTr + ASTP + BLKP + STLP + TOVP' \
                       ' + C(height_dummies) + rookie + youth + oldness + standard_salary + std_salary_resid + Wins + ' \
                'team_match + C(Race) * demean_clinton + C(Race) * demean_race_diff + total_population + C(Race) * demean_PPG',
                data = nba_df,
               weights = 1,# / (nba_df['compound_mean_std'] / np.sqrt(fit_df['user_count'])),
#                missing='raise'   
               ).fit(cov_type='cluster', cov_kwds={'groups' : nba_df['Player']})

In [51]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:          compound_mean   R-squared:                       0.002
Model:                            WLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     5.780
Date:                Mon, 24 Dec 2018   Prob (F-statistic):           1.25e-17
Time:                        08:58:24   Log-Likelihood:            -2.7978e+05
No. Observations:              880544   AIC:                         5.596e+05
Df Residuals:                  880517   BIC:                         5.599e+05
Df Model:                          26                                         
Covariance Type:              cluster                                         
=================================================================================================
                                    coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept                         0.0810      0.018      4.430      0.000       0.045       0.117
C(height_dummies)[T.short]        0.0113      0.010      1.127      0.260      -0.008       0.031
C(height_dummies)[T.tall]        -0.0019      0.005     -0.383      0.702      -0.012       0.008
rookie[T.True]                    0.0161      0.005      3.007      0.003       0.006       0.027
team_match[T.True]                0.0023      0.002      1.296      0.195      -0.001       0.006
C(Race)[T.W]                      0.0167      0.010      1.610      0.107      -0.004       0.037
MP                            -2.957e-06   3.26e-06     -0.906      0.365   -9.35e-06    3.44e-06
ThreePP                           0.0106      0.015      0.727      0.467      -0.018       0.039
DWS                               0.0013      0.002      0.571      0.568      -0.003       0.006
TRBP                             -0.0011      0.001     -2.282      0.023      -0.002      -0.000
FTr                              -0.0017      0.009     -0.190      0.850      -0.020       0.016
ASTP                             -0.0002      0.000     -0.804      0.421      -0.001       0.000
BLKP                              0.0019      0.001      1.537      0.124      -0.001       0.004
STLP                              0.0022      0.002      1.063      0.288      -0.002       0.006
TOVP                             -0.0003      0.000     -1.050      0.294      -0.001       0.000
youth                             0.0025      0.001      2.678      0.007       0.001       0.004
oldness                           0.0015      0.001      2.264      0.024       0.000       0.003
standard_salary                7.694e-05      0.000      0.268      0.789      -0.000       0.001
std_salary_resid              -5.014e-05      0.000     -0.172      0.864      -0.001       0.001
Wins                          -4.294e-05      0.000     -0.282      0.778      -0.000       0.000
demean_clinton                    0.0062      0.003      1.811      0.070      -0.001       0.013
C(Race)[T.W]:demean_clinton       0.0030      0.010      0.306      0.760      -0.016       0.022
demean_race_diff              -1.704e-05   4.18e-05     -0.407      0.684    -9.9e-05    6.49e-05
C(Race)[T.W]:demean_race_diff  6.386e-05      0.000      0.621      0.534      -0.000       0.000
total_population              -6.446e-11   1.41e-10     -0.458      0.647    -3.4e-10    2.12e-10
demean_PPG                        0.0010      0.001      1.868      0.062   -4.91e-05       0.002
C(Race)[T.W]:demean_PPG           0.0024      0.001      2.248      0.025       0.000       0.004
==============================================================================
Omnibus:                     1066.833   Durbin-Watson:          

## NFL

### z_DVOA only

In [45]:
nfl_df['demean_DVOA'] = nfl_df['z_DVOA'] - nfl_df['z_DVOA'].mean()
nfl_df['demean_clinton'] = nfl_df['clinton_vote_lead'] - nfl_df['clinton_vote_lead'].mean()
nfl_df['demean_race_diff'] = nfl_df['white_black_diff'] - nfl_df['white_black_diff'].mean()

In [23]:
nfl_model = smf.wls( formula = 'compound_mean ~ z_DVOA',#\
                       #' + white_black_diff * C(race) + clinton_vote_lead',',#'
                data = nfl_df, weights = 1 
               ).fit(cov_type='cluster', cov_kwds={'groups' : nfl_df['Player']})

In [24]:
nfl_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:          compound_mean   R-squared:                       0.000
Model:                            WLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     9.575
Date:                Mon, 24 Dec 2018   Prob (F-statistic):            0.00210
Time:                        08:34:59   Log-Likelihood:                -96293.
No. Observations:              292531   AIC:                         1.926e+05
Df Residuals:                  292529   BIC:                         1.926e+05
Df Model:                           1                                         
Covariance Type:              cluster                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0670      0.002     31.603      0.000       0.063       0.071
z_DVOA         0.0057      0.002      3.094      0.002       0.002       0.009
==============================================================================
Omnibus:                      237.333   Durbin-Watson:                   1.957
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              263.144
Skew:                          -0.031   Prob(JB):                     7.23e-58
Kurtosis:                       3.133   Cond. No.                         1.46
==============================================================================

Warnings:
[1] Standard Errors are robust tocluster correlation (cluster)
"""

### Performance + demo + position + salary

In [32]:
nfl_model = smf.wls( formula = 'compound_mean ~ z_DVOA + youth + oldness + rookie + C(position) + C(height_dummies) + Wins + standard_salary + std_salary_resid',#\
                       #' + white_black_diff * C(race) + clinton_vote_lead',',#'
                data = nfl_df, weights = 1, 
               ).fit(cov_type='cluster', cov_kwds={'groups' : nfl_df['Player']})

In [33]:
nfl_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:          compound_mean   R-squared:                       0.001
Model:                            WLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     5.489
Date:                Mon, 24 Dec 2018   Prob (F-statistic):           3.16e-08
Time:                        08:39:40   Log-Likelihood:                -96157.
No. Observations:              292531   AIC:                         1.923e+05
Df Residuals:                  292519   BIC:                         1.925e+05
Df Model:                          11                                         
Covariance Type:              cluster                                         
=============================================================================================
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept                     0.0683      0.067      1.014      0.310      -0.064       0.200
rookie[T.True]                0.0172      0.007      2.316      0.021       0.003       0.032
C(position)[T.rb]             0.0077      0.006      1.273      0.203      -0.004       0.020
C(position)[T.te]            -0.0060      0.005     -1.188      0.235      -0.016       0.004
C(position)[T.wr]            -0.0063      0.006     -0.976      0.329      -0.019       0.006
C(height_dummies)[T.tall]     0.0041      0.006      0.727      0.468      -0.007       0.015
z_DVOA                        0.0074      0.002      3.963      0.000       0.004       0.011
youth                         0.0009      0.002      0.542      0.588      -0.002       0.004
oldness                      -0.0006      0.001     -0.764      0.445      -0.002       0.001
Wins                         -0.0001      0.001     -0.205      0.838      -0.001       0.001
standard_salary              -0.0004      0.004     -0.104      0.917      -0.009       0.008
std_salary_resid           7.146e-05      0.003      0.021      0.983      -0.006       0.007
==============================================================================
Omnibus:                      239.797   Durbin-Watson:                   1.958
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              265.519
Skew:                          -0.032   Prob(JB):                     2.20e-58
Kurtosis:                       3.133   Cond. No.                         879.
==============================================================================

Warnings:
[1] Standard Errors are robust tocluster correlation (cluster)
"""

### Performance + demo + position + salary + race

In [38]:
nfl_model = smf.wls( formula = 'compound_mean ~ z_DVOA + youth + oldness + rookie + C(position) + C(height_dummies) + Wins + standard_salary + std_salary_resid'\
                       '+ C(race)',#' + white_black_diff *  + clinton_vote_lead',',#'
                data = nfl_df, weights = 1, 
               ).fit(cov_type='cluster', cov_kwds={'groups' : nfl_df['Player']})

In [39]:
nfl_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:          compound_mean   R-squared:                       0.001
Model:                            WLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     5.241
Date:                Mon, 24 Dec 2018   Prob (F-statistic):           2.88e-08
Time:                        08:42:56   Log-Likelihood:                -96152.
No. Observations:              292531   AIC:                         1.923e+05
Df Residuals:                  292518   BIC:                         1.925e+05
Df Model:                          12                                         
Covariance Type:              cluster                                         
=============================================================================================
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept                     0.0667      0.067      0.995      0.320      -0.065       0.198
rookie[T.True]                0.0172      0.007      2.310      0.021       0.003       0.032
C(position)[T.rb]             0.0117      0.006      1.818      0.069      -0.001       0.024
C(position)[T.te]            -0.0058      0.005     -1.120      0.263      -0.016       0.004
C(position)[T.wr]            -0.0028      0.007     -0.403      0.687      -0.016       0.011
C(height_dummies)[T.tall]     0.0037      0.006      0.649      0.517      -0.007       0.015
C(race)[T.W]                  0.0058      0.006      1.025      0.306      -0.005       0.017
z_DVOA                        0.0072      0.002      3.891      0.000       0.004       0.011
youth                         0.0011      0.002      0.659      0.510      -0.002       0.004
oldness                      -0.0006      0.001     -0.818      0.413      -0.002       0.001
Wins                         -0.0001      0.001     -0.248      0.805      -0.001       0.001
standard_salary              -0.0006      0.004     -0.148      0.883      -0.009       0.008
std_salary_resid           4.982e-05      0.003      0.015      0.988      -0.006       0.007
==============================================================================
Omnibus:                      239.514   Durbin-Watson:                   1.958
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              265.232
Skew:                          -0.032   Prob(JB):                     2.54e-58
Kurtosis:                       3.133   Cond. No.                         880.
==============================================================================

Warnings:
[1] Standard Errors are robust tocluster correlation (cluster)
"""

### Performance + demo + position + salary + race + user + interactions

In [46]:
nfl_model = smf.wls( formula = 'compound_mean ~ C(race) * demean_DVOA + youth + oldness + rookie + C(position) + C(height_dummies) + Wins + standard_salary + std_salary_resid'\
                       '+ C(race) * demean_race_diff  + C(race) * demean_clinton + total_population',
                data = nfl_df, weights = 1, 
               ).fit(cov_type='cluster', cov_kwds={'groups' : nfl_df['Player']})

In [47]:
nfl_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:          compound_mean   R-squared:                       0.001
Model:                            WLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     6.238
Date:                Mon, 24 Dec 2018   Prob (F-statistic):           1.08e-13
Time:                        08:49:06   Log-Likelihood:                -96137.
No. Observations:              292531   AIC:                         1.923e+05
Df Residuals:                  292512   BIC:                         1.925e+05
Df Model:                          18                                         
Covariance Type:              cluster                                         
=================================================================================================
                                    coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept                         0.0724      0.066      1.094      0.274      -0.057       0.202
C(race)[T.W]                      0.0066      0.006      1.179      0.238      -0.004       0.018
rookie[T.True]                    0.0177      0.007      2.464      0.014       0.004       0.032
C(position)[T.rb]                 0.0119      0.006      1.834      0.067      -0.001       0.025
C(position)[T.te]                -0.0069      0.005     -1.269      0.204      -0.017       0.004
C(position)[T.wr]                -0.0025      0.007     -0.374      0.708      -0.016       0.011
C(height_dummies)[T.tall]         0.0038      0.006      0.668      0.504      -0.007       0.015
demean_DVOA                       0.0051      0.004      1.444      0.149      -0.002       0.012
C(race)[T.W]:demean_DVOA          0.0039      0.004      0.989      0.323      -0.004       0.012
youth                             0.0011      0.002      0.680      0.497      -0.002       0.004
oldness                          -0.0007      0.001     -0.952      0.341      -0.002       0.001
Wins                             -0.0002      0.001     -0.434      0.664      -0.001       0.001
standard_salary                  -0.0007      0.004     -0.169      0.866      -0.009       0.007
std_salary_resid                 -0.0002      0.003     -0.070      0.945      -0.007       0.006
demean_race_diff               5.831e-06   9.32e-05      0.063      0.950      -0.000       0.000
C(race)[T.W]:demean_race_diff     0.0002      0.000      1.442      0.149   -6.42e-05       0.000
demean_clinton                    0.0043      0.006      0.773      0.440      -0.007       0.015
C(race)[T.W]:demean_clinton       0.0139      0.007      1.912      0.056      -0.000       0.028
total_population              -3.284e-10   1.77e-10     -1.853      0.064   -6.76e-10    1.89e-11
==============================================================================
Omnibus:                      238.327   Durbin-Watson:                   1.958
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              264.004
Skew:                          -0.031   Prob(JB):                     4.70e-58
Kurtosis:                       3.133   Cond. No.                     3.26e+08
==============================================================================

Warnings:
[1] Standard Errors are robust tocluster correlation (cluster)
[2] The condition number is large, 3.26e+08. This might indicate that there are
strong multicollinearity or other numerical problems.
"""